In [2]:
import requests
from bs4 import BeautifulSoup
import json
from time import time

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

/Users/Welcome/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [22]:
#utils.py part
TYPE_CONFIG = {
    "사회": "society",
    "정치": "politics",
    "경제": "economic",
    "국제": "foreign",
    "문화": "culture",
    "연예": "entertain",
    "스포츠": "sports",
    "IT": "digital",
    "칼럼": "editorial",
    "보도자료": "press",
}

In [45]:
#**kwars 이거써서 코드 깔끔하게
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

def makeURL(search = "", period="", start_date="", end_date="") -> str:
    url = "https://search.daum.net/search?w=news&DA=STC&enc=utf8"
    period_url = ""
    search_url = ""
    
    if period != "": #[TODO] 올바른 period가 안들어왔을때 에러 핸들링
        period_url = "&period=%s" % (period)
        
        if period == "u": #기간 조정
            period_url = period_url + "&sd=%s&ed=%s" % (start_date, end_date)
    
    if search !="":
        search_url = "&q=%s" % (search)
    
    return url + period_url + search_url

def newsScrape(search):
    URL = makeURL(search)
    count = 1
    for page_number in range(1, 81):
        try:
            original_html = requests.get(URL + "&page=" + str(page_number), headers = headers)
            html = BeautifulSoup(original_html.text, "html.parser")
            print(html)
            title_list = html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
            if len(title_list) == 0:
                break
            else:
                count = count + 1
                print(count)
        except(error):
            print("Error from news scrape function", error)
            break

def task(url) -> bool:
    try:
        original_html = requests.get(url)
        #parsing and getting title
        html = BeautifulSoup(original_html.text, "html.parser")
        title_list = html.select("div.box_etc > ul.list_news2.list_allnews > li div.cont_thumb > strong.tit_thumb > a")
        
        if len(title_list) ==0:
            return False
        
    except(error):
        print("Error from newsType function", error, count)
        return False
    
    finally:
        return True
        
def newsType(newsType="", date=""):
    THREAD_COUNT = 10
    URL = "https://news.daum.net/breakingnews/"
    if newsType !="":
        URL = URL + TYPE_CONFIG[newsType] #[TODO] TYPE_CONFIG에 들어있지 않는 타입 들어왔을때 에러 핸들링
    
    if date != "":  #[TODO] 올바른 date가 아닐때 에러 핸들링
        URL = URL + "?regDate=" + date
        
    count = 1
    with ThreadPoolExecutor(THREAD_COUNT) as executor:
        while(True):
            url = URL + "&page=" + str(count)
            task_return = executor.submit(task, url)
            print(task_return)
            count = count + 1
            if count > 10:
                break

if __name__ == '__main__':        
    start = time()
    newsType("경제", "20220720")
    print("time :", time() - start)
    #newsScrape("우영우")

<Future at 0x10d21a340 state=running>
<Future at 0x10ce41a00 state=running>
<Future at 0x10ce41e20 state=running>
<Future at 0x10d2044c0 state=running>
<Future at 0x10d6918e0 state=running>
<Future at 0x10d3a98b0 state=running>
<Future at 0x10d61e0a0 state=running>
<Future at 0x10d3fc2e0 state=running>
<Future at 0x10d3fcbe0 state=running>
<Future at 0x10d3ee520 state=running>
time : 0.6510539054870605
